In [2]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import time
import json
from collections import namedtuple
import csv

In [2]:
def cleaner(r, u):
    try:
        c = r[0].p.get_text(strip=True)
    except:
        c = ''
    try:
        d = r[3].img['src']
    except:
        d = ''
    return  [u+r[0].a['href'], r[0].a.get_text(strip=True), c, r[1].get_text(strip=True), r[2].get_text(strip=True), d]

In [3]:
classes = ['company-description', 'company-contact', 'company-info']

def cleaner_(r):
    vals = [l.a['href'] for l in r[1].find_all('li', class_ = 'actions')]
    a = [v.split(':')[1] if v[0] in ['c', 'm']  else v for v in vals]
    b = r[0].get_text(strip=True)
    c = '\n'.join([d.get_text() for d in r[1].address.find_all('div')][1:])
    r2 = r[2].find_all('div', recursive = False)
    vals_ = [a_ for a_ in r2[1].find_all('li')]
    e = []
    for v_ in vals_:
        v = unicode(v_.h5.text)
        v_.h5.clear()
        e.append((v, " ".join(v_.get_text(strip=True).split())))
    try:
        r2[2]
    except IndexError:
        d = []
    else:
        d = [(a_.get_text(), a_.a['href']) for a_ in r2[2].find_all(class_ = 'file')]
    return [b]+a+[c]+e+d

In [4]:
companies = []
for p in range(0, 17):
    url = 'http://www.swedishlifesciences.se'
    ext = "/company-index?title=&page="+str(int(p))
    print url+ext
    soup = BeautifulSoup(requests.get(url+ext).text, "html.parser")
    data = soup.table
    companies = companies+[cleaner(rows.find_all('td'), url) for rows in data.find_all('tr', class_ = "company")]
    time.sleep(10)    

http://www.swedishlifesciences.se/company-index?title=&page=0
http://www.swedishlifesciences.se/company-index?title=&page=1
http://www.swedishlifesciences.se/company-index?title=&page=2
http://www.swedishlifesciences.se/company-index?title=&page=3
http://www.swedishlifesciences.se/company-index?title=&page=4
http://www.swedishlifesciences.se/company-index?title=&page=5
http://www.swedishlifesciences.se/company-index?title=&page=6
http://www.swedishlifesciences.se/company-index?title=&page=7
http://www.swedishlifesciences.se/company-index?title=&page=8
http://www.swedishlifesciences.se/company-index?title=&page=9
http://www.swedishlifesciences.se/company-index?title=&page=10
http://www.swedishlifesciences.se/company-index?title=&page=11
http://www.swedishlifesciences.se/company-index?title=&page=12
http://www.swedishlifesciences.se/company-index?title=&page=13
http://www.swedishlifesciences.se/company-index?title=&page=14
http://www.swedishlifesciences.se/company-index?title=&page=15
ht

In [ ]:
companies_full = []
print len(companies)
for i, c in enumerate(companies):
    try:
        if not i % 10:
            print i
        url = c[0]
        soup = BeautifulSoup(requests.get(url).text, "html.parser")
        data = soup.find(class_  = 'main-content')
        if len(data.find_all('div')) == 3:
            companies_full.append(c+[data.find_all('div')[2].a['href']])
        else:
            companies_full.append(c+['']+cleaner_(data.find_all(class_ = classes)))
        time.sleep(10)
    except:
        print c[0]

In [11]:
path = '../data/dad_data.csv'
with open(path, 'wb') as f:
    w = csv.writer(f)
    w.writerow(['URL', 'NAME', 'MOTTO', 'CITY', 'CATEGORY', 'LOGO_LINK', 'COMPANY_WEBSITE', 'OTHER_DATA'])
    w.writerows([unicode(c_).encode("utf-8") for c_ in c] for c in companies_full)

In [64]:
url = 'http://www.biotechumea.se/'
ext = 'pharmaceuticals-and-diagnostics'
soup = BeautifulSoup(requests.get(url+ext).text, "html.parser")
data = soup.find_all('div', class_ = 'item-list company-list')[1]

In [65]:
comp_list = [[l.text, url+l.a['href']] for l in data.find_all('li')]

In [138]:
comp_list_full = []
for c in comp_list:
    try:
        soup = BeautifulSoup(requests.get(c[1]).text, "html.parser")
        data = soup.find('div', class_ = 'CS_article')
        a = [d.text for d in data.find_all(['h1', 'div'], recursive=False)[:-1]]+[d.get_text(strip=True) for d in data.find_all('div', recursive=False)[-1].find_all(['p', 'div'], recursive=False)]
        comp_list_full = comp_list_full+[c+a]
    except:
        comp_list_full = comp_list_full+[c]
        print c[1] 

http://www.biotechumea.se/essum 
http://www.biotechumea.se/helicure 
http://www.biotechumea.se/inrobiomedtek 
http://www.biotechumea.se/it-instrumentteknik 
http://www.biotechumea.se/pelagia-miljokonsult 
http://www.biotechumea.se/techtum-lab 


In [139]:
path = '../data/dad_data_2.csv'
with open(path, 'wb') as f:
    w = csv.writer(f)
    w.writerow(['NAME', 'LINK', 'DESCRIPTION', 'DESCRIPTION2', 'CONTACT', 'CONTACT_DETAILS'])
    w.writerows([unicode(c_).encode("utf-8") for c_ in c] for c in comp_list_full)